In [1]:
print("📦 Installing required packages...")
print("This will take 2-3 minutes...\n")

# Core ML libraries
!pip install -q transformers torch torchvision

# Data handling
!pip install -q gcsfs pandas numpy pillow

# Progress bars
!pip install -q tqdm

# Downgrade numpy if needed (for scipy compatibility)
# !pip install -q 'numpy==1.26.4'

print("\n✅ All packages installed!\n")

📦 Installing required packages...
This will take 2-3 minutes...


✅ All packages installed!



In [2]:
# ============================================================================
# CELL 1: NOTEBOOK HEADER (Markdown Cell)
# ============================================================================
"""
# Amazon ML Challenge 2025 - Phase 2: Vision Transformer Features

**Objective:** Extract 768-dimensional visual features from product images using pre-trained Vision Transformer

**Input:** 150,000 product images (75k train + 75k test) from GCS bucket

**Output:**
- `train_vit_features.npy` - Shape: (75000, 768) - Size: ~230 MB
- `test_vit_features.npy` - Shape: (75000, 768) - Size: ~230 MB

**Model:** google/vit-base-patch16-224 (86M parameters, Apache 2.0 license)

**Timeline:** 3-4 hours total processing time

**Status:** Phase 0 EDA Complete ✅ | Phase 2 In Progress 🔄

**Start Time:** Saturday, October 11, 2025 - 1:31 PM IST
"""


# ============================================================================
# CELL 2: CONFIGURATION
# ============================================================================

# GCS Configuration (from Phase 0 EDA)
GCS_BUCKET = "amazon_ml_challenge_data"
TRAIN_IMAGE_PATH = f"gs://{GCS_BUCKET}/dataset/images/train"
TEST_IMAGE_PATH = f"gs://{GCS_BUCKET}/dataset/images/test"
# TRAIN_CSV_PATH = f"gs://{GCS_BUCKET}/dataset/train.csv"
# TEST_CSV_PATH = f"gs://{GCS_BUCKET}/dataset/test.csv"
TRAIN_CSV_PATH = "/content/train.csv"
TEST_CSV_PATH = "/content/test.csv"

# Processing Configuration
BATCH_SIZE = 32  # Process 32 images at once (optimal for T4 GPU)
IMAGE_SIZE = 224  # ViT input size (224x224 pixels)
FEATURE_DIM = 768  # ViT output dimension

# Known Issues from EDA
MISSING_TRAIN_SAMPLE_ID = 279285  # Image failed to download
# Test missing ID unknown - will handle automatically

print("="*80)
print("🔬 PHASE 2: VISION TRANSFORMER FEATURE EXTRACTION")
print("="*80)
print(f"📦 GCS Bucket: {GCS_BUCKET}")
print(f"🖼️  Batch Size: {BATCH_SIZE} images")
print(f"📐 Image Size: {IMAGE_SIZE}x{IMAGE_SIZE} pixels")
print(f"🎯 Feature Dimension: {FEATURE_DIM}")
print("="*80 + "\n")

🔬 PHASE 2: VISION TRANSFORMER FEATURE EXTRACTION
📦 GCS Bucket: amazon_ml_challenge_data
🖼️  Batch Size: 32 images
📐 Image Size: 224x224 pixels
🎯 Feature Dimension: 768



In [3]:


# ============================================================================
# CELL 4: Import Libraries
# ============================================================================

import os
import pandas as pd
import numpy as np
import gcsfs
from PIL import Image
import io
import time
from tqdm.auto import tqdm
import warnings
warnings.filterwarnings('ignore')

# PyTorch
import torch
from torch.utils.data import Dataset, DataLoader

# Transformers
from transformers import AutoImageProcessor, AutoModel

print("✅ Libraries imported!\n")

# Check GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🖥️  Device: {device}")

if device.type == 'cuda':
    print(f"   GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"   GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
    print(f"   CUDA Version: {torch.version.cuda}")
else:
    print("   ⚠️  WARNING: No GPU detected!")
    print("   Go to Runtime → Change runtime type → Select GPU")

print()



✅ Libraries imported!

🖥️  Device: cuda
   GPU Name: Tesla T4
   GPU Memory: 15.8 GB
   CUDA Version: 12.6



In [5]:
from google.colab import auth
auth.authenticate_user()

In [6]:
# ============================================================================
# CELL 5: Connect to GCS and Load CSVs
# ============================================================================

print("🔗 Connecting to Google Cloud Storage...")

# Initialize GCS filesystem
fs = gcsfs.GCSFileSystem()

# Test connection
try:
    test_files = fs.ls(f"{GCS_BUCKET}/dataset")
    print("✅ GCS connection successful!\n")
except Exception as e:
    print(f"❌ GCS connection failed: {e}")
    print("Make sure you have access to the bucket!\n")

# Load CSVs
print("📖 Loading CSV files...")

with open(TRAIN_CSV_PATH, 'rb') as f:
    df_train = pd.read_csv(f)
print(f"✅ Train CSV loaded: {len(df_train):,} rows")

with open(TEST_CSV_PATH, 'rb') as f:
    df_test = pd.read_csv(f)
print(f"✅ Test CSV loaded: {len(df_test):,} rows\n")

🔗 Connecting to Google Cloud Storage...
✅ GCS connection successful!

📖 Loading CSV files...
✅ Train CSV loaded: 75,000 rows
✅ Test CSV loaded: 75,000 rows



In [9]:
#  ============================================================================
# CELL 6: Load Vision Transformer Model
# ============================================================================

print("🤖 LOADING VISION TRANSFORMER MODEL")
print("="*80)
print("Model: google/vit-base-patch16-224")
print("Parameters: 86M")
print("License: Apache 2.0 ✅")
print("Output: 768-dimensional feature vector")
print("="*80 + "\n")

print("Downloading model weights (~340 MB)...")
print("This happens once, then cached...\n")

start_time = time.time()

# Load image processor (handles preprocessing)
vit_processor = AutoImageProcessor.from_pretrained("google/vit-base-patch16-224", use_fast=True)
print("✅ Image processor loaded")

# Load model
vit_model = AutoModel.from_pretrained("google/vit-base-patch16-224")
print("✅ Model architecture loaded")

# Move model to GPU
vit_model = vit_model.to(device)
vit_model.eval()  # Set to evaluation mode (no training)
print(f"✅ Model moved to {device}")

load_time = time.time() - start_time

print(f"\n✅ Vision Transformer ready! ({load_time:.1f}s)")
print(f"   Input size: {IMAGE_SIZE}x{IMAGE_SIZE} pixels")
print(f"   Output size: {FEATURE_DIM} features\n")


🤖 LOADING VISION TRANSFORMER MODEL
Model: google/vit-base-patch16-224
Parameters: 86M
License: Apache 2.0 ✅
Output: 768-dimensional feature vector

This happens once, then cached...



Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ Image processor loaded
✅ Model architecture loaded
✅ Model moved to cuda

✅ Vision Transformer ready! (0.9s)
   Input size: 224x224 pixels
   Output size: 768 features



In [10]:
# ============================================================================
# CELL 7: Define Custom Dataset Class
# ============================================================================

class ProductImageDataset(Dataset):
    """
    Custom PyTorch Dataset for loading product images from GCS.

    Handles:
    - Loading images from GCS bucket
    - Missing images (returns zero vector)
    - Corrupted images (returns zero vector)
    - Image preprocessing for ViT
    """

    def __init__(self, df, image_base_path, fs, processor, image_size=224):
        """
        Args:
            df: DataFrame with sample_id column
            image_base_path: GCS path to image folder
            fs: gcsfs filesystem object
            processor: ViT image processor
            image_size: Target image size (224 for ViT)
        """
        self.df = df.reset_index(drop=True)
        self.image_base_path = image_base_path
        self.fs = fs
        self.processor = processor
        self.image_size = image_size

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        """
        Load and preprocess a single image.

        Returns:
            dict with:
            - 'pixel_values': preprocessed image tensor
            - 'sample_id': sample ID
            - 'status': 'success' or 'missing'
        """
        sample_id = self.df.iloc[idx]['sample_id']

        # Construct image path
        image_path = f"{self.image_base_path}/{sample_id}.jpg"

        try:
            # Load image from GCS
            with self.fs.open(image_path, 'rb') as f:
                image_data = f.read()

            # Open with PIL
            image = Image.open(io.BytesIO(image_data))

            # Convert to RGB (handles RGBA, grayscale, etc.)
            image = image.convert('RGB')

            # Preprocess with ViT processor
            inputs = self.processor(images=image, return_tensors="pt")

            return {
                'pixel_values': inputs['pixel_values'].squeeze(0),
                'sample_id': sample_id,
                'status': 'success'
            }

        except Exception as e:
            # Image not found or corrupted
            # Return zero tensor (will become zero vector after ViT)
            zero_image = torch.zeros(3, self.image_size, self.image_size)

            return {
                'pixel_values': zero_image,
                'sample_id': sample_id,
                'status': 'missing'
            }

print("✅ Dataset class defined!\n")


✅ Dataset class defined!



In [11]:
# ============================================================================
# CELL 8: Feature Extraction Function
# ============================================================================

def extract_vit_features(dataloader, model, device, dataset_name="Dataset"):
    """
    Extract ViT features for entire dataset using batched processing.

    Args:
        dataloader: PyTorch DataLoader
        model: ViT model
        device: cuda or cpu
        dataset_name: Name for progress bar

    Returns:
        features: numpy array of shape (N, 768)
        metadata: DataFrame with sample_id, status, processing_time
    """
    model.eval()

    all_features = []
    all_metadata = []

    print(f"🔬 Extracting features for {dataset_name}...")
    print(f"   Total batches: {len(dataloader)}")
    print(f"   Batch size: {dataloader.batch_size}")
    print(f"   Expected time: ~{len(dataloader) * 1.7 / 60:.0f} minutes\n")

    start_time = time.time()

    with torch.no_grad():  # No gradient computation (faster)
        for batch_idx, batch in enumerate(tqdm(dataloader, desc=f"Processing {dataset_name}", ncols=100)):
            batch_start = time.time()

            # Move batch to GPU
            pixel_values = batch['pixel_values'].to(device)
            sample_ids = batch['sample_id']
            statuses = batch['status']

            # Forward pass through ViT
            outputs = model(pixel_values=pixel_values)

            # Extract pooled features (768-dim per image)
            features = outputs.pooler_output  # Shape: (batch_size, 768)

            # Move to CPU and convert to numpy
            features_np = features.cpu().numpy()

            # Store features
            all_features.append(features_np)

            # Store metadata
            batch_time = time.time() - batch_start
            for i in range(len(sample_ids)):
                all_metadata.append({
                    'sample_id': sample_ids[i].item(),
                    'status': statuses[i],
                    'batch_idx': batch_idx,
                    'processing_time': batch_time / len(sample_ids)
                })

            # Clear GPU cache every 100 batches
            if (batch_idx + 1) % 100 == 0:
                torch.cuda.empty_cache()

    # Concatenate all features
    features_array = np.vstack(all_features)

    # Create metadata DataFrame
    metadata_df = pd.DataFrame(all_metadata)

    # Calculate statistics
    elapsed_time = time.time() - start_time
    images_per_sec = len(metadata_df) / elapsed_time

    print(f"\n✅ Feature extraction complete!")
    print(f"   Features shape: {features_array.shape}")
    print(f"   Time taken: {elapsed_time/60:.1f} minutes")
    print(f"   Speed: {images_per_sec:.1f} images/second")

    # Count missing images
    missing_count = (metadata_df['status'] == 'missing').sum()
    success_count = (metadata_df['status'] == 'success').sum()

    print(f"\n   Success: {success_count:,} ({100*success_count/len(metadata_df):.2f}%)")
    print(f"   Missing: {missing_count:,} ({100*missing_count/len(metadata_df):.2f}%)")

    if missing_count > 0:
        missing_ids = metadata_df[metadata_df['status'] == 'missing']['sample_id'].tolist()
        if len(missing_ids) <= 10:
            print(f"   Missing sample IDs: {missing_ids}")
        else:
            print(f"   Missing sample IDs (first 10): {missing_ids[:10]}")

    print()

    return features_array, metadata_df

print("✅ Feature extraction function defined!\n")

✅ Feature extraction function defined!



In [ ]:
# ============================================================================
# CELL 9: Process TRAIN Dataset
# ============================================================================

print("\n" + "="*80)
print("🎯 PROCESSING TRAIN DATASET")
print("="*80 + "\n")

# Create dataset
train_dataset = ProductImageDataset(
    df=df_train,
    image_base_path=TRAIN_IMAGE_PATH,
    fs=fs,
    processor=vit_processor,
    image_size=IMAGE_SIZE
)

# Create dataloader
train_dataloader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,  # Keep original order
    num_workers=2,  # Parallel data loading
    pin_memory=True  # Faster GPU transfer
)

print(f"📊 Train dataset prepared:")
print(f"   Total images: {len(train_dataset):,}")
print(f"   Batches: {len(train_dataloader):,}")
print(f"   Batch size: {BATCH_SIZE}\n")

# Extract features
train_features, train_metadata = extract_vit_features(
    dataloader=train_dataloader,
    model=vit_model,
    device=device,
    dataset_name="TRAIN"
)

# Verify shape
assert train_features.shape == (75000, 768), f"Expected (75000, 768), got {train_features.shape}"
print("✅ Shape verification passed!\n")

# Feature statistics
print("📊 TRAIN FEATURE STATISTICS:")
print(f"   Mean: {np.mean(train_features):.4f}")
print(f"   Std: {np.std(train_features):.4f}")
print(f"   Min: {np.min(train_features):.4f}")
print(f"   Max: {np.max(train_features):.4f}")
print(f"   NaN count: {np.isnan(train_features).sum()}")
print(f"   Inf count: {np.isinf(train_features).sum()}\n")


🎯 PROCESSING TRAIN DATASET

📊 Train dataset prepared:
   Total images: 75,000
   Batches: 2,344
   Batch size: 32

🔬 Extracting features for TRAIN...
   Total batches: 2344
   Batch size: 32
   Expected time: ~66 minutes



Processing TRAIN:   0%|                                                    | 0/2344 [00:00<?, ?it/s]

In [ ]:
# ============================================================================
# CELL 10: Process TEST Dataset
# ============================================================================

print("\n" + "="*80)
print("🎯 PROCESSING TEST DATASET")
print("="*80 + "\n")

# Create dataset
test_dataset = ProductImageDataset(
    df=df_test,
    image_base_path=TEST_IMAGE_PATH,
    fs=fs,
    processor=vit_processor,
    image_size=IMAGE_SIZE
)

# Create dataloader
test_dataloader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=2,
    pin_memory=True
)

print(f"📊 Test dataset prepared:")
print(f"   Total images: {len(test_dataset):,}")
print(f"   Batches: {len(test_dataloader):,}")
print(f"   Batch size: {BATCH_SIZE}\n")

# Extract features
test_features, test_metadata = extract_vit_features(
    dataloader=test_dataloader,
    model=vit_model,
    device=device,
    dataset_name="TEST"
)

# Verify shape
assert test_features.shape == (75000, 768), f"Expected (75000, 768), got {test_features.shape}"
print("✅ Shape verification passed!\n")

# Feature statistics
print("📊 TEST FEATURE STATISTICS:")
print(f"   Mean: {np.mean(test_features):.4f}")
print(f"   Std: {np.std(test_features):.4f}")
print(f"   Min: {np.min(test_features):.4f}")
print(f"   Max: {np.max(test_features):.4f}")
print(f"   NaN count: {np.isnan(test_features).sum()}")
print(f"   Inf count: {np.isinf(test_features).sum()}\n")


In [ ]:
# ============================================================================
# CELL 11: Save Features to Local Storage
# ============================================================================

print("💾 SAVING FEATURES...")
print("="*80 + "\n")

# Save train features
train_features_path = '/content/train_vit_features.npy'
np.save(train_features_path, train_features)
train_size_mb = os.path.getsize(train_features_path) / (1024**2)
print(f"✅ Saved: train_vit_features.npy")
print(f"   Location: {train_features_path}")
print(f"   Size: {train_size_mb:.1f} MB")
print(f"   Shape: {train_features.shape}\n")

# Save test features
test_features_path = '/content/test_vit_features.npy'
np.save(test_features_path, test_features)
test_size_mb = os.path.getsize(test_features_path) / (1024**2)
print(f"✅ Saved: test_vit_features.npy")
print(f"   Location: {test_features_path}")
print(f"   Size: {test_size_mb:.1f} MB")
print(f"   Shape: {test_features.shape}\n")

# Save metadata
train_metadata_path = '/content/train_vit_metadata.csv'
train_metadata.to_csv(train_metadata_path, index=False)
print(f"✅ Saved: train_vit_metadata.csv")
print(f"   Location: {train_metadata_path}\n")

test_metadata_path = '/content/test_vit_metadata.csv'
test_metadata.to_csv(test_metadata_path, index=False)
print(f"✅ Saved: test_vit_metadata.csv")
print(f"   Location: {test_metadata_path}\n")

print(f"📊 Total size: {train_size_mb + test_size_mb:.1f} MB\n")


# ============================================================================
# CELL 12: Mount Google Drive and Backup
# ============================================================================

print("☁️  BACKING UP TO GOOGLE DRIVE...")
print("="*80 + "\n")

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Create output directory
drive_output_dir = '/content/drive/MyDrive/Amazon_ML_2025/phase2_image_features'
os.makedirs(drive_output_dir, exist_ok=True)

print(f"📁 Output directory: {drive_output_dir}\n")

# Copy files to Drive
import shutil

print("Copying files to Google Drive...")

# Copy train features
shutil.copy(train_features_path, f"{drive_output_dir}/train_vit_features.npy")
print("✅ train_vit_features.npy copied")

# Copy test features
shutil.copy(test_features_path, f"{drive_output_dir}/test_vit_features.npy")
print("✅ test_vit_features.npy copied")

# Copy metadata
shutil.copy(train_metadata_path, f"{drive_output_dir}/train_vit_metadata.csv")
print("✅ train_vit_metadata.csv copied")

shutil.copy(test_metadata_path, f"{drive_output_dir}/test_vit_metadata.csv")
print("✅ test_vit_metadata.csv copied")

print(f"\n✅ All files backed up to Google Drive!")
print(f"   Location: {drive_output_dir}\n")


# ============================================================================
# CELL 13: Final Summary and Next Steps
# ============================================================================

print("\n" + "="*80)
print("🎉 PHASE 2 COMPLETE - VISION TRANSFORMER FEATURES EXTRACTED!")
print("="*80 + "\n")

print("📊 SUMMARY:")
print(f"   Train features: {train_features.shape} - {train_size_mb:.1f} MB")
print(f"   Test features: {test_features.shape} - {test_size_mb:.1f} MB")
print(f"   Total processing time: ~{(time.time() - start_time)/3600:.1f} hours")
print(f"   Missing train images: {(train_metadata['status'] == 'missing').sum()}")
print(f"   Missing test images: {(test_metadata['status'] == 'missing').sum()}")

print("\n✅ FILES SAVED:")
print("   Local (temporary):")
print(f"      {train_features_path}")
print(f"      {test_features_path}")
print("   Google Drive (permanent):")
print(f"      {drive_output_dir}/train_vit_features.npy")
print(f"      {drive_output_dir}/test_vit_features.npy")

print("\n🎯 NEXT STEPS:")
print("="*80)
print("Phase 3: TEXT FEATURE EXTRACTION")
print("   A. Sentence Embeddings (384-dim)")
print("   B. Hand-Crafted REGEX Features (~20 features)")
print("   C. TF-IDF (Optional)")
print("\nPhase 4: FEATURE COMBINATION")
print("   - Combine image + text features")
print("   - Apply log transform to target")
print("   - Create train/val split")
print("\nPhase 5: MODEL TRAINING")
print("   - XGBoost + LightGBM + CatBoost")
print("   - Ensemble predictions")
print("   - Generate submission")
print("="*80)

print("\n🚀 Ready for Phase 3!")
print("⏰ Current time:", time.strftime("%I:%M %p"))
print("📅 Date:", time.strftime("%B %d, %Y"))